<img src="image.png" style="height:250px">

# Import

In [15]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

dataset = pd.read_csv('50_Startups.csv')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, 4].values

In [ ]:
X

# Preparing Data

In [16]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
lbencoder = LabelEncoder()
ohencoder = OneHotEncoder(categorical_features=[3])

X[:, 3] = lbencoder.fit_transform(X[:, 3])
X = ohencoder.fit_transform(X).toarray()

/home/vibhu/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/vibhu/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:390: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [ ]:
X

## Avoiding Dummy Data Trap

In [17]:
X = X[:, 1:]

## Splitting into test and training

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Linear Regression

In [19]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()

In [20]:
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [21]:
y_pred = regressor.predict(X_test)

In [ ]:
y_pred

In [ ]:
y_test

# Backward Elimination

<img src="bw elimination.png" style="height:350px">

## Handling the constant term

In [22]:
X = np.append(np.ones((50,1)).astype(int), values=X, axis=1)

In [ ]:
X

## Backward Elimination

In [23]:
import statsmodels.formula.api as sm

In [24]:
significance_level = 0.05

In [25]:
lst = [0,1,2,3,4,5]
while len(lst)>0:
    X_opt = X[:, lst]
    regressor_OLS = sm.OLS(y,X_opt).fit()
    print(regressor_OLS.summary())
    i = regressor_OLS.pvalues.argmax()
    if regressor_OLS.pvalues[i] <= significance_level:
        break
    else:
        print('\nRemoved column -->',lst[i])
        del lst[i]

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.945
Method:                 Least Squares   F-statistic:                     169.9
Date:                Sun, 06 Jan 2019   Prob (F-statistic):           1.34e-27
Time:                        17:43:51   Log-Likelihood:                -525.38
No. Observations:                  50   AIC:                             1063.
Df Residuals:                      44   BIC:                             1074.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5.013e+04   6884.820      7.281      0.0

In [ ]:
regressor_OLS.summary()

In [26]:
X_opt = X_opt[:, 1:]

In [27]:
X_train_OLS, X_test_OLS, y_train, y_test = train_test_split(X_opt, y, test_size = 0.2, random_state = 0)

In [28]:
regressor.fit(X_train_OLS, y_train)
y_pred_OLS = regressor.predict(X_test_OLS)

In [29]:
for a,b,c in zip(y_pred_OLS,y_test,y_pred):
    print(a,'  ',b,'  ',c)

104667.27805997859    103282.38    103015.20159796352
134150.8341057847    144259.4    132582.2776081537
135207.80019516923    146121.95    132447.73845174702
72170.54428855634    77798.83    71976.09851258088
179090.58602508352    191050.39    178537.48221055616
109824.77386586336    105008.31    116161.24230165542
65644.27773756774    81229.06    67851.69209675616
100481.43277138655    97483.56    98791.73374687255
111431.75202432103    110352.25    113969.43533012968
169438.148435391    166187.94    167921.06569550867


In [30]:
from sklearn.metrics import r2_score
print(r2_score(y_test,y_pred_OLS))
print(r2_score(y_test,y_pred))

0.9464587607787219
0.9347068473282446
